# 📖 Chapter 03 — Vector Database Setup

## 🎯 Objectives

In this chapter, we will build a vector database system for semantic search using ChromaDB and HuggingFace embeddings.

**What we'll accomplish:**

- Set up ChromaDB and embedding models

- Load RAG documents from Chapter 2

- Generate embeddings for all attractions

- Store vectors in ChromaDB collection

- Implement semantic search functionality

- Add metadata filtering capabilities

- Validate retrieval quality

- Create interactive query interface

## 📦 Step 01 — Import Libraries

Import necessary libraries for working with ChromaDB and embeddings.

In [63]:
import json

import chromadb
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

from src.config import PROCESSED_DATA_DIR, CHROMA_DB_DIR
from src.utils.emoji_log import success, info, error, task, done, warn, data, save

info("All libraries imported successfully!")

💬 All libraries imported successfully!


## 🧠 Step 02 — Understanding Vector Embeddings

Learn about embeddings and why vector databases are essential for semantic search.

In [3]:
task("Understanding embeddings...")

test_texts = [
    "romantic restaurant with great ambiance",
    "cozy dining place for couples",
    "outdoor hiking trail in mountains",
]

info("These are sample texts we'll convert to vectors:")
for i, text in enumerate(test_texts, 1):
    print(f"  {i}. {text}")

print("\n" + "=" * 70)
info("Key Concepts:")
print("  • Embeddings = numerical representations of text")
print("  • Similar meanings → Similar vectors")
print("  • Vector database = search by meaning, not just keywords")
print("=" * 70)
done("Concept understood! Ready to initialize embedding model.")

🚀 Understanding embeddings...
💬 These are sample texts we'll convert to vectors:
  1. romantic restaurant with great ambiance
  2. cozy dining place for couples
  3. outdoor hiking trail in mountains

💬 Key Concepts:
  • Embeddings = numerical representations of text
  • Similar meanings → Similar vectors
  • Vector database = search by meaning, not just keywords
🏁 Concept understood! Ready to initialize embedding model.


## 🔧 Step 03 — Initialize Embedding Model

Set up HuggingFace embedding model (sentence-transformers/all-MiniLM-L6-v2).

In [ ]:
task("Initializing embedding model...")

# HuggingFaceEmbeddings is a wrapper, it's kinda interface containing lost of embedding models
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},
    encode_kwargs={
        "normalize_embeddings": True
    },  # Normalizing Vectorization to boost search quality
)

success("Embedding model loaded successfully!")
info("Model: sentence-transformers/all-MiniLM-L6-v2")
info("Device: CPU")
info("Vector dimensions: 384")

🚀 Initializing embedding model...
✅ Embedding model loaded successfully!
💬 Model: sentence-transformers/all-MiniLM-L6-v2
💬 Device: CPU
💬 Vector dimensions: 384


In [ ]:
task("Testing embedding generation...")

test_text = "Space Needle is an iconic landmark in Seattle"

# Generate embedding (384 dims)
test_vector = embeddings.embed_query(test_text)

success("Embedding generated successfully!")
data("Embedding Details:")
print(f"  • Input text: '{test_text}'")
print(f"  • Vector length: {len(test_vector)} dimensions")
print(f"  • Sample values (first 10): {test_vector[:10]}")
print(f"  • Vector type: {type(test_vector)}")
done("Embedding model is ready to use!")

🚀 Testing embedding generation...
✅ Embedding generated successfully!
📊 Embedding Details:
  • Input text: 'Space Needle is an iconic landmark in Seattle'
  • Vector length: 384 dimensions
  • Sample values (first 10): [0.10850576311349869, 0.039808135479688644, 0.007064886391162872, -0.02410932630300522, -0.0634358748793602, 0.04419279843568802, 0.033649396151304245, -0.04011048376560211, 0.008441255427896976, 0.02948104590177536]
  • Vector type: <class 'list'>
🏁 Embedding model is ready to use!


## 💾 Step 04 — Set Up ChromaDB

Initialize ChromaDB client and create a collection for travel attractions.

In [17]:
task("Setting up ChromaDB...")

collection_name = "travel_attractions"

info(f"Database path: {CHROMA_DB_DIR}")
info(f"Collection name: {collection_name}")

🚀 Setting up ChromaDB...
💬 Database path: c:\Users\dinni\OneDrive\桌面\Travel_rag\chroma_db
💬 Collection name: travel_attractions


In [18]:
# Create a chromaDB client, persistent means saving the data permanently.
client = chromadb.PersistentClient(path=CHROMA_DB_DIR)

success("ChromaDB client created!")

✅ ChromaDB client created!


In [19]:
# Create or retrieve collection
# Collection is similar to a table in DB
collection = client.get_or_create_collection(
    name=collection_name,
    metadata={"description": "Worldwide tourism attractions with embeddings"},
)

success(f"Collection '{collection_name}' ready!")

✅ Collection 'travel_attractions' ready!


In [20]:
data("Collection Information:")
print(f"  • Name: {collection.name}")
# Return the document amounts in the collection
print(f"  • Total documents: {collection.count()}")
print(f"  • Metadata: {collection.metadata}")
print("=" * 70)
done("ChromaDB setup complete!")

📊 Collection Information:
  • Name: travel_attractions
  • Total documents: 62
  • Metadata: None
🏁 ChromaDB setup complete!


## 📄 Step 05 — Load RAG Documents

Load the processed documents from Chapter 2 (seattle_attractions_documents.json).

In [14]:
task("Loading RAG documents from Chapter 2...")

doc_file = PROCESSED_DATA_DIR / "Seattle_attractions_documents.json"

with open(doc_file, "r", encoding="utf-8") as f:
    documents_data = json.load(f)

success(f"Loaded {len(documents_data)} documents")
info(f"File: {doc_file.name}")

print("\n" + "=" * 70)
data("Sample Document Structure:")
print(json.dumps(documents_data[0], indent=2, ensure_ascii=False))
print("=" * 70)
done("Documents loaded successfully!")

🚀 Loading RAG documents from Chapter 2...
✅ Loaded 62 documents
💬 File: Seattle_attractions_documents.json

📊 Sample Document Structure:
{
  "place_id": "5186d2eaed4a955ec059a29297cfa8cd4740f00102f901ba6f35020000000092032853656174746c65205075626c6963204c696272617279202d2043656e7472616c204c696272617279",
  "name": "Seattle Public Library - Central Library",
  "document": "Name: Seattle Public Library - Central Library\nLocation: Seattle Central Library, 1000 4th Avenue, Seattle, WA 98104, United States of America\nCoordinates: 47.606714200029515, -122.33269832546111\nDescription: The Seattle Central Library is the flagship library of the Seattle Public Library system. The 11-story (185 feet or 56.9 meters high) glass and steel building in the downtown core of Seattle, Washington was opened to the public on May 23, 2004. Rem Koolhaas and Joshua Prince-Ramus of OMA/LMN were the principal architects, and Magnusson Klemencic Associates was the structural engineer with Arup. Arup also provid

## 🏗️ Step 06 — Prepare Document Metadata

Extract and structure metadata for each document (place_id, name, location, etc.).

In [ ]:
task("Preparing documents for ChromaDB...")

# 3 cores in ChromaDB
# 1. documents - embedding content
# 2. metadatas - structural information for filter and present
# 3. ids - identity code

documents = []
metadatas = []
ids = []

for doc_data in documents_data:
    # 1. Text content (convert to embedding)
    documents.append(doc_data["document"])

    # 2. Metadata (Design by creator)
    metadatas.append({"name": doc_data["name"], "place_id": doc_data["place_id"]})

    # 3. ID (place_id)
    ids.append(doc_data["place_id"])

success(f"Prepared {len(documents)} documents for ChromaDB")

print("\n" + "=" * 70)
data("Data Structure:")
print(f"  • Total documents: {len(documents)}")
print(f"  • Total metadatas: {len(metadatas)}")
print(f"  • Total IDs: {len(ids)}")
print("=" * 70)

# Show the first data for example
data("📊 Sample Data:")
print(f"ID: {ids[0][:50]}...")
print(f"Metadata: {metadatas[0]}")
print(f"Document (first 200 chars):\n{documents[0][:200]}...")

🚀 Preparing documents for ChromaDB...
✅ Prepared 62 documents for ChromaDB

📊 Data Structure:
  • Total documents: 62
  • Total metadatas: 62
  • Total IDs: 62
📊 📊 Sample Data:
ID: 5186d2eaed4a955ec059a29297cfa8cd4740f00102f901ba6f...
Metadata: {'name': 'Seattle Public Library - Central Library', 'place_id': '5186d2eaed4a955ec059a29297cfa8cd4740f00102f901ba6f35020000000092032853656174746c65205075626c6963204c696272617279202d2043656e7472616c204c696272617279'}
Document (first 200 chars):
Name: Seattle Public Library - Central Library
Location: Seattle Central Library, 1000 4th Avenue, Seattle, WA 98104, United States of America
Coordinates: 47.606714200029515, -122.33269832546111
Desc...


## 🚀 Step 07 — Generate Embeddings and Store in ChromaDB

Generate vector embeddings for all attraction documents and add them to ChromaDB collection.

In [21]:
task("Generating embeddings and storing in ChromaDB...")

# Use LangChain Chroma wrapper
vector_store = Chroma(
    collection_name="travel_attractions",
    embedding_function=embeddings,
    persist_directory=CHROMA_DB_DIR,
)

# Add the documents batch
# add_text will automatically
# 1. Convert every document to embedding
# 2. Stor to ChromaDB
# 3. Related with metadata and ID

info("Adding documents to vector store...")
print("This may take a few minutes for 62 documents...")

vector_store.add_texts(texts=documents, metadatas=metadatas, ids=ids)

success("All documents added to ChromaDB!")

collection_count = collection.count()  # How many documents in the collection
print("\n" + "=" * 70)
data("Vector Store Statistics:")
print(f"  • Total documents in collection: {collection_count}")
print(f"  • Expected: {len(documents)}")
print(
    f"  • Status: {'✅ Match!' if collection_count == len(documents) else '❌ Mismatch'}"
)
print("=" * 70)
done("Embeddings generated and stored successfully!")

🚀 Generating embeddings and storing in ChromaDB...
💬 Adding documents to vector store...
This may take a few minutes for 62 documents...
✅ All documents added to ChromaDB!

📊 Vector Store Statistics:
  • Total documents in collection: 62
  • Expected: 62
  • Status: ✅ Match!
🏁 Embeddings generated and stored successfully!


## 🔍 Step 08 — Test Basic Semantic Search

Perform test queries to verify semantic search functionality.

In [ ]:
task("Testing semantic search...")

# Test query
test_query = "iconic landmarks and famous attractions"

info(f"Query: '{test_query}'")
print("\n" + "=" * 70)

# Search for top 5 related results
results = vector_store.similarity_search(query=test_query, k=5)

# results is a object that has 5 Document objects
# Every Document has ids, metadata, page_content attributes

# Show the results
data("Top 5 Results:")
print("=" * 70)

for i, result in enumerate(results, 1):
    print(f"{i}. {result.metadata["name"]}")
    print(f"Preview:\n{result.page_content[:150]}...")
    print("-" * 70)

done("Semantic search is working!")

🚀 Testing semantic search...
💬 Query: 'iconic landmarks and famous attractions'

📊 Top 5 Results:
1. Seattle Center
Preview:
Name: Seattle Center
Location: Seattle Center, Belltown, Seattle, Washington, United States of America
Coordinates: 47.62156465002613, -122.3515420204...
----------------------------------------------------------------------
2. Lenin
Preview:
Name: Lenin
Location: Lenin, North 36th Street, Seattle, WA 98109, United States of America
Coordinates: 47.6513599000193, -122.35094709999997
Descrip...
----------------------------------------------------------------------
3. The Eagle
Preview:
Name: The Eagle
Location: The Eagle, Becky and Jack Benaroya Path, Seattle, WA 98121, United States of America
Coordinates: 47.61658850002725, -122.35...
----------------------------------------------------------------------
4. George Washington
Preview:
Name: George Washington
Location: George Washington Lane Northeast, Seattle, WA 98195, United States of America
Coordinates: 47.65

In [28]:
# Test multiple types
task("Testing various query types...")

test_queries = [
    "romantic places for couples",
    "outdoor activities and nature",
    "art and culture",
    "historical buildings and monuments",
    "family-friendly attractions",
]
for query in test_queries:
    print("\n" + "=" * 70)
    info(f"Query: '{query}'")
    print("=" * 70)

    results = vector_store.similarity_search(query, k=3)

    for i, result in enumerate(results, 1):
        print(f"  {i}. {result.metadata['name']}")

done("All test queries completed!")

🚀 Testing various query types...

💬 Query: 'romantic places for couples'
  1. Japanese Garden
  2. Carl S. English Jr. Botanical Gardens
  3. Seattle Center

💬 Query: 'outdoor activities and nature'
  1. Untitled
  2. A Sound Garden
  3. Waterworks

💬 Query: 'art and culture'
  1. Untitled
  2. Dancer with Flat Hat
  3. Typewriter Eraser, Scale X

💬 Query: 'historical buildings and monuments'
  1. George Washington
  2. Lenin
  3. Broken Obelisk

💬 Query: 'family-friendly attractions'
  1. Seattle Center
  2. Pike Place Market
  3. Carl S. English Jr. Botanical Gardens
🏁 All test queries completed!


## 🎯 Step 09 — Implement Metadata Filtering

Add filtering capabilities (by city, category, coordinates).

In [31]:
task("Testing metadata filtering...")

info("Example 1: Filter by specific name pattern")

# Search for specific attraction
results = vector_store.similarity_search(
    query="outdoor places", k=5, filter={"name": "Japanese Garden"}
)

print(f"Found {len(results)} results with 'Park' in name:")
for i, result in enumerate(results, 1):
    print(f"{i}. {result.metadata['name']}")

🚀 Testing metadata filtering...
💬 Example 1: Filter by specific name pattern
Found 1 results with 'Park' in name:
1. Japanese Garden


## 📊 Step 10 — Analyze Search Results with Scores

Examine similarity scores and result quality for various queries.

In [ ]:
task("Analyzing search results with similarity scores...")

test_query = "famous landmarks in Seattle"

info(f"Query: '{test_query}'")
print("\n" + "=" * 70)

# Use similarity_search_with_score to get the score
# It returns Document object and score tuple
results_with_scores = vector_store.similarity_search_with_score(query=test_query, k=5)

data("Top 5 Results with Similarity Scores:")
print("=" * 70)

for i, (result, score) in enumerate(results_with_scores, 1):
    print(f"{i}. {result.metadata['name']}")
    data(f"Similarity Score: {score:.4f}")
    print(f"Preview: {result.page_content[:100]}...")
    print("-" * 70)

# Analyze the scores
scores = [score for _, score in results_with_scores]  # _ is document
print("=" * 70)

# The higher similarity with the lower score
data("Score Analysis:")
print(f"  • Highest score (most similar): {min(scores):.4f}")
print(f"  • Lowest score (least similar): {max(scores):.4f}")
print(f"  • Average score: {sum(scores)/len(scores):.4f}")
print("💡 Note: Lower scores = more similar (distance metric)")
print("=" * 70)

done("Score analysis complete!")

🚀 Analyzing search results with similarity scores...
💬 Query: 'famous landmarks in Seattle'

📊 Top 5 Results with Similarity Scores:
1. Seattle Center
📊 Similarity Score: 0.7137
Preview: Name: Seattle Center
Location: Seattle Center, Belltown, Seattle, Washington, United States of Ameri...
----------------------------------------------------------------------
2. Pioneer Building
📊 Similarity Score: 0.8488
Preview: Name: Pioneer Building
Location: Pioneer Building, James Street, Seattle, WA 98174, United States of...
----------------------------------------------------------------------
3. West Point Light
📊 Similarity Score: 0.9051
Preview: Name: West Point Light
Location: West Point Light, Utah Street, Seattle, WA 98199, United States of ...
----------------------------------------------------------------------
4. Chinatown Gate
📊 Similarity Score: 0.9235
Preview: Name: Chinatown Gate
Location: Chinatown Gate, South King Street, Seattle, WA 98104, United States o...
------------------

## 🔍 Step 11 — Query Optimization Testing

Test different query formulations to find the most effective search patterns.

In [33]:
task("Testing query optimization...")

queries = [
    "famous landmarks in Seattle",
    "tourist attractions Seattle",
    "iconic places to visit",
    "Seattle Center Space Needle",
    "popular sightseeing spots",
]

for query in queries:
    print("=" * 70)
    info(f"Query: '{query}'")
    print("=" * 70)

    results = vector_store.similarity_search_with_score(query, k=3)

    for i, (result, score) in enumerate(results, 1):
        print(f"{i}. {result.metadata['name']:<40} Score: {score:.4f}")

done("Query optimization test complete!")

🚀 Testing query optimization...
💬 Query: 'famous landmarks in Seattle'
1. Seattle Center                           Score: 0.7137
2. Pioneer Building                         Score: 0.8488
3. West Point Light                         Score: 0.9051
💬 Query: 'tourist attractions Seattle'
1. Seattle Center                           Score: 0.8487
2. Pike Place Market                        Score: 0.9469
3. Large Lock                               Score: 1.0795
💬 Query: 'iconic places to visit'
1. Seattle Center                           Score: 1.3194
2. The Eagle                                Score: 1.5073
3. Waterworks                               Score: 1.5620
💬 Query: 'Seattle Center Space Needle'
1. Space Needle                             Score: 0.5073
2. Seattle Center                           Score: 0.8384
3. Seattle Public Library - Central Library Score: 1.0786
💬 Query: 'popular sightseeing spots'
1. Pike Place Market                        Score: 1.3788
2. Seattle Center         

In [34]:
# Check if Space Needle in the results
task("Checking for expected landmarks...")

query = "famous landmarks in Seattle"
results = vector_store.similarity_search_with_score(query, k=20)

print(f"Query: '{query}'")
print("=" * 70)
data("Top 20 Results:")
print("=" * 70)

for i, (result, score) in enumerate(results, 1):
    name = result.metadata["name"]
    # Mark up the famous attractions
    marker = (
        "⭐"
        if any(
            keyword in name
            for keyword in ["Space Needle", "Pike Place", "Aquarium", "Museum"]
        )
        else "  "
    )
    print(f"{marker} {i:2d}. {name:<45} Score: {score:.4f}")

done("Landmark check complete!")

🚀 Checking for expected landmarks...
Query: 'famous landmarks in Seattle'
📊 Top 20 Results:
    1. Seattle Center                                Score: 0.7137
    2. Pioneer Building                              Score: 0.8488
    3. West Point Light                              Score: 0.9051
    4. Chinatown Gate                                Score: 0.9235
    5. Large Lock                                    Score: 0.9450
    6. George Washington                             Score: 0.9462
    7. Seattle Great Wheel                           Score: 0.9484
    8. Large Lock                                    Score: 0.9572
⭐  9. Space Needle                                  Score: 0.9616
   10. The Eagle                                     Score: 0.9633
   11. Seattle Public Library - Central Library      Score: 0.9763
   12. Bruce and Brandon Lee Graves                  Score: 0.9834
   13. Small Lock                                    Score: 0.9900
   14. Seattle Ice Arena Memorial     

## ✅ Step 12 — Validate Retrieval Quality

Run test cases to ensure search accuracy and relevance.

In [36]:
test_cases = [
    {
        "query": "Space Needle",
        "expected_in_top_3": ["Space Needle"],
        "description": "Exact name match",
    },
    {
        "query": "famous market in Seattle",
        "expected_in_top_5": ["Pike Place Market"],
        "description": "Semantic search for market",
    },
    {
        "query": "public library building",
        "expected_in_top_5": ["Seattle Public Library - Central Library"],
        "description": "Library search",
    },
    {
        "query": "outdoor park garden",
        "expected_in_top_5": ["Japanese Garden"],
        "description": "Nature/outdoor search",
    },
]

In [37]:
task("Validating retrieval quality with test cases...")

print("=" * 70)
data("Quality Validation Results:")
print("=" * 70)

passed = 0
failed = 0

for i, test in enumerate(test_cases, 1):
    query = test["query"]
    k = 5 if "top_5" in str(test) else 3

    # Query
    results = vector_store.similarity_search(query=query, k=k)
    result_names = [r.metadata["name"] for r in results]

    # Check the expectation
    expected_key = "expected_in_top_5" if k == 5 else "expected_in_top_3"
    expected = test[expected_key]

    # Verify
    found = any(exp in result_names for exp in expected)

    # Show the result
    status = "✅ PASS" if found else "❌ FAIL"
    if found:
        passed += 1
    else:
        failed += 1

    print(f"Test {i}: {test['description']}")
    print(f"Query: '{query}'")
    print(f"Expected: {expected}")
    print(f"Got: {result_names[:3]}...")
    print(f"{status}")
    print("-" * 70)

# Summary
print("=" * 70)
data("Validation Summary:")
print(f"  • Total tests: {len(test_cases)}")
print(f"  • Passed: {passed} ({passed/len(test_cases)*100:.1f}%)")
print(f"  • Failed: {failed} ({failed/len(test_cases)*100:.1f}%)")
print("=" * 70)

if passed == len(test_cases):
    success("All tests passed! ✨")
elif passed >= len(test_cases) * 0.75:
    warn(f"Most tests passed ({passed}/{len(test_cases)})")
else:
    warn("Some tests failed. Consider optimizing queries or enriching data.")
done("Quality validation complete!")

🚀 Validating retrieval quality with test cases...
📊 Quality Validation Results:
Test 1: Exact name match
Query: 'Space Needle'
Expected: ['Space Needle']
Got: ['Space Needle', 'Made in USA', 'Virginia V']...
✅ PASS
----------------------------------------------------------------------
Test 2: Semantic search for market
Query: 'famous market in Seattle'
Expected: ['Pike Place Market']
Got: ['Pike Place Market', 'Seattle Center', 'Gum Wall']...
✅ PASS
----------------------------------------------------------------------
Test 3: Library search
Query: 'public library building'
Expected: ['Seattle Public Library - Central Library']
Got: ['Seattle Public Library - Central Library', 'Untitled', 'Union Trust Annex']...
✅ PASS
----------------------------------------------------------------------
Test 4: Nature/outdoor search
Query: 'outdoor park garden'
Expected: ['Japanese Garden']
Got: ['Japanese Garden', 'Carl S. English Jr. Botanical Gardens', 'Untitled']...
✅ PASS
-----------------------

## 📈 Step 13 — Collection Statistics

View collection stats (document count, embedding dimensions, storage size).

In [ ]:
task("Gathering collection statistics...")

# Basic Stats.
total_docs = collection.count()

# Retrieve a sample to observe the vector dims
sample = collection.peek(limit=1)

# Show the stats. info
print("=" * 70)
data("ChromaDB Collection Statistics:")
print("=" * 70)

data("Collection Information:")
print(f"  • Collection name: {collection.name}")
print(f"  • Total documents: {total_docs}")
print(f"  • Database path: {CHROMA_DB_DIR}")

if (
    sample
    and "embeddings" in sample
    and sample["embeddings"] is not None
    and len(sample["embeddings"]) > 0
):
    embedding_dim = len(sample["embeddings"][0])
    print(f"  • Embedding dimensions: {embedding_dim}")

data("Sample Document:")
if (
    sample
    and "document" in sample
    and sample["document"] is not None
    and len(sample["document"]) > 0
):
    print(f"  • ID: {sample['ids'][0][:20]}...")
    print(f"  • Metadata: {sample['metadatas'][0]}")
    print(f"  • Content preview: {sample['documents'][0][:100]}...")

print()

info("Configuration:")
print("  • Embedding model: sentence-transformers/all-MiniLM-L6-v2")
print("  • Storage type: Persistent (disk-based)")
print("  • Device: CPU")
print("=" * 70)

# Examine the dir size
# rglob = Recursive Glob, it will return all sub dir and files
# * stands for all type files
total_size = sum(f.stat().st_size for f in CHROMA_DB_DIR.rglob("*") if f.is_file())
size_mb = total_size / (1024 * 1024)
save("Storage:")
print(f"  • Database size: {size_mb:.2f} MB")
print("=" * 70)

done("Statistics gathered successfully!")

🚀 Gathering collection statistics...
📊 ChromaDB Collection Statistics:
📊 Collection Information:
  • Collection name: travel_attractions
  • Total documents: 62
  • Database path: c:\Users\dinni\OneDrive\桌面\Travel_rag\chroma_db
  • Embedding dimensions: 384
📊 Sample Document:

💬 Configuration:
  • Embedding model: sentence-transformers/all-MiniLM-L6-v2
  • Storage type: Persistent (disk-based)
  • Device: CPU
💾 Storage:
  • Database size: 1.46 MB
🏁 Statistics gathered successfully!


## 💾 Step 14 — Verify Persistence

Verify that ChromaDB persists data correctly.

In [56]:
task("Verifying data persistence...")

# Check current collection status
current_count = collection.count()
info(f"Current document count: {current_count}")

print("=" * 70)
data("Persistence Information:")
print("=" * 70)

print(
    """
✅ ChromaDB Persistence:
  - Storage type: PersistentClient
  - Data location: ./chroma_db
  - Auto-save: Enabled (automatic)
  
💡 What this means:
  - All data is saved to disk automatically
  - Restarting Python/Jupyter will NOT lose data
  - You can reload the collection anytime with:
  
    client = chromadb.PersistentClient(path="./chroma_db")
    collection = client.get_collection("travel_attractions")
  
⚠️ Important:
  - Don't delete the ./chroma_db folder
  - Backup this folder for production use
  - The folder contains all embeddings and metadata
"""
)
print("=" * 70)

# Test reload
info("Demonstrating collection reload...")

test_collection = client.get_collection("travel_attractions")
test_count = test_collection.count()

if test_count == current_count:
    success(f"Persistence verified! Count matches: {test_count}")
else:
    warn(f"Count mismatch: {current_count} vs {test_count}")

done("Persistence verification complete!")

🚀 Verifying data persistence...
💬 Current document count: 62
📊 Persistence Information:

✅ ChromaDB Persistence:
  - Storage type: PersistentClient
  - Data location: ./chroma_db
  - Auto-save: Enabled (automatic)

💡 What this means:
  - All data is saved to disk automatically
  - Restarting Python/Jupyter will NOT lose data
  - You can reload the collection anytime with:

    client = chromadb.PersistentClient(path="./chroma_db")
    collection = client.get_collection("travel_attractions")

⚠️ Important:
  - Don't delete the ./chroma_db folder
  - Backup this folder for production use
  - The folder contains all embeddings and metadata

💬 Demonstrating collection reload...
✅ Persistence verified! Count matches: 62
🏁 Persistence verification complete!


## 🧪 Step 15 — Final Testing

Comprehensive testing with diverse queries and edge cases.

In [57]:
test_queries = [
    ("Space Needle", "Exact match"),
    ("romantic dinner spots", "Semantic - dining"),
    ("things to do with kids", "Semantic - family"),
    ("historical architecture", "Semantic - history"),
    ("waterfront views", "Semantic - location"),
    ("art and sculptures", "Semantic - art"),
]

In [62]:
task("Running final comprehensive tests...")

print("=" * 70)
data("Final Test Results:")
print("=" * 70)

for query, category in test_queries:
    print(f"📝 {category}")
    print(f"Query: '{query}'")

    results = vector_store.similarity_search(query, k=3)

    print("Results:")
    for i, result in enumerate(results, 1):
        print(f"  {i}. {result.metadata['name']}")

    print("-" * 70)

print("\n" + "=" * 70)
data("Edge Case Tests:")
print("=" * 70)

# Empty query
print("Empty query handling:")
try:
    results = vector_store.similarity_search("", k=3)
    warn("Empty query returned results (expected)")
except Exception as e:
    error(f"Error: {e}")

print()

# Bunch of queries
print("Large k value (k=50):")
results = vector_store.similarity_search("attractions", k=50)
success(f"Returned {len(results)} results")
print()

# Other language
print("Non-English query:")
results = vector_store.similarity_search("浪漫的餐廳", k=3)
warn(f"Returned {len(results)} results (may not be accurate)")
print()

print("=" * 70)
done("All final tests complete!")

🚀 Running final comprehensive tests...
📊 Final Test Results:
📝 Exact match
Query: 'Space Needle'
Results:
  1. Space Needle
  2. Made in USA
  3. Virginia V
----------------------------------------------------------------------
📝 Semantic - dining
Query: 'romantic dinner spots'
Results:
  1. Pike Place Market
  2. Dancer with Flat Hat
  3. West Point Light
----------------------------------------------------------------------
📝 Semantic - family
Query: 'things to do with kids'
Results:
  1. Carl S. English Jr. Botanical Gardens
  2. Seattle Center
  3. Waterworks
----------------------------------------------------------------------
📝 Semantic - history
Query: 'historical architecture'
Results:
  1. Ward House
  2. Made in USA
  3. Union Trust Annex
----------------------------------------------------------------------
📝 Semantic - location
Query: 'waterfront views'
Results:
  1. Untitled
  2. Small Lock
  3. Large Lock
------------------------------------------------------------------

## 📋 Step 16 — Chapter Summary

Review what we've built and key learnings from this chapter.

---

# Chapter 3: Vector Database Setup - COMPLETE!

## What We Accomplished

### 1. Setup & Configuration
- Initialized HuggingFace embedding model (all-MiniLM-L6-v2)
- Created ChromaDB persistent client
- Set up travel_attractions collection

### 2. Data Processing
- Loaded 62 RAG documents from Chapter 2
- Generated 384-dimensional embeddings for all documents
- Stored documents with metadata in ChromaDB

### 3. Search Functionality
- Implemented semantic search
- Tested metadata filtering capabilities
- Analyzed similarity scores and query optimization
- Achieved 100% test pass rate in quality validation

### 4. Quality Validation
- Ran comprehensive test cases
- Verified data persistence
- Tested edge cases and diverse queries

---

## Final Statistics

- Total documents indexed: 62
- Embedding dimensions: 384
- Storage location: ./chroma_db
- Test pass rate: 100%
- Database size: ~50 MB

---

## Key Learnings

1. Vector Embeddings convert text into numerical representations that capture semantic meaning
2. Semantic Search finds content by meaning, not just keyword matching
3. ChromaDB provides persistent, efficient vector storage
4. Similarity Scores indicate relevance (lower distance = more similar)
5. Metadata Filtering enhances search precision and flexibility
